## Problema de las n reinas

- Programación avanzada
- Olima, Natalia

### Librerías

In [ ]:
import numpy as np
import random as rd
import matplotlib.pyplot as plt
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
from matplotlib import colors

### Código del algoritmo

In [ ]:
class ReinasN:
    '''Clase del algoritmo Reinas N que contiene todo el proceso'''
    def __init__(self, n):
        self.n = n
    
    def generar_pob(self, n):
        '''Genera la población inicial aleatoriamente con 20 individuos'''
        poblacion = []
        for i in range(20):
            pob_ini = np.random.randint(0, self.n, size = self.n).tolist()
            poblacion.append(pob_ini)
        return self.resolucion(n, poblacion, 0)
    
    def resolucion(self,n, pob, cont):
        '''Función recursiva que somete a la población al algoritmo genético (la primer población es 
        la generada por comenzar_proceso y las siguientes son las mutadas)'''
        if not type(pob[:]) == list:
            return 'Programa finalizado' #Para que se detenga el programa después de generar el tablero
        else: 
            cont +=1
            while cont <=2000: 
                pob = self.ajustar(pob)
                return self.resolucion(n, pob, cont)#Utiliza la recursividad para realizar el proceso 
                                                #con las siguientes generaciones
    
    def ajustar(self, poblacion):
        '''Somete a la población al algorítmo genético.
        Compara a cada 'gen' del individuo con los demás. Agrega la puntuación cada individuo a una lista de selección.
        A mayor puntuación más amenazas entre reinas, puntuación ideal = 0 (sin amenazas). Cada amenaza suma 1 punto.'''
        lista_puntuacion = []
        for p in poblacion:
            lista_1 = []
            for j,i in enumerate(p):
                for l,k in enumerate(p):
                    if j == l: #Si es el mismo no hacer nada
                        break 
                    #i fila, j columna, k otra fila, l otra columna
                    #Se fija si hay reinas en la misma diagonal o fila
                    #Si  hay, entonces hay amenaza y agrega un 1
                    if ((p[j] - j == p[l] - l or p[j] + j == p[l] + l) or (j - p[l] == p[j] - p[l] or j-l == p[l]-p[j])):
                        lista_1.append(1) #1 amenaza
                    elif p[j]==p[l]: #Si están en la misma fila (equivalente a decir que los números son iguales)
                        lista_1.append(1) #1 amenaza
                    #Si no hay amenaza, entonces agrega un 0
                    else:
                        lista_1.append(0) #0 no amenaza
            lista_puntuacion.append(sum(lista_1))     
        return self.seleccionar(lista_puntuacion, poblacion) #Pasa a la etapa de selección
    
    def seleccionar(self, lista, pob):
        '''Selecciona a los 10 individuos más aptos. Da la prioridad según la puntuación'''
        seleccionados = [] #Guarda los 10 individuos seleccionados
        for j,i in enumerate(lista): #Si la puntuación es cero, el individuo es óptimo
            if i == 0:
                return self.imprimir_optimo(pob[j])
        for l in range(len(lista)): #Los va agregando por órden de puntuación
            l -= 1
            for j,i in enumerate(lista):
                if i in range(l+1,l+2) and len(seleccionados)<10:
                    seleccionados.append(pob[j])
        return self.cruzar(seleccionados) #Pasa a la etapa de cruce
    
    def cruzar(self, pob_sel):
        '''Cruza a los individuos aptos para crear nuevos con caracerísticas similares. 
        Los corta por la mitad y los combina unos con otros'''
        pob_cruzada = []
        lista_aux_pa = []
        lista_aux_pb = []
        if self.n % 2 == 0: #si n es par
            for p in pob_sel:
                lista_aux_pa.append(p[0:int(len(p)/2)+1]) #Guarda la mitad en una lista
                lista_aux_pb.append(p[int(len(p)/2)+1:]) #Y la otra mitad en otra
            for j,a in enumerate(lista_aux_pa): #combina las mitades
                for i in range(int(len(a)-j+1)):
                    pob_cruzada.append(a + lista_aux_pb[j+i+1])   
        else: #si n es impar
            for p in pob_sel: 
                lista_aux_pa.append(p[0:int(len(p)//2+1)]) #Guarda la mitad menos 1 en una lista 
                lista_aux_pb.append(p[int(len(p)//2)+1:]) #Y la otra mitad mas 1 en otra
            for j,a in enumerate(lista_aux_pa):#combina las mitades
                for i in range(int(len(a)-j+2)):
                    pob_cruzada.append(a + lista_aux_pb[j+i+1])
        return self.mutar(pob_cruzada)
    
    def mutar(self, pob):
        '''Muta a los individuos para modificarlos.
        A cada individuo le desordena los genes aleatoriamiente.
        Además, si el primer gen está entre 0 y la longitud-1 entonces le suma 1 de lo contrario le resta 1.
        Si el gen que se encuentra en el medio está entre 0 y la longitud -1 le resta 1'''
        nueva_generacion = []
        for p in pob:  
            rd.shuffle(p) #Desordena los genes del individuo
            if 0<=p[0]<self.n-1: #Suma 1 
                p[0] = p[0]+1
            else:
                p[0] = p[0]-1 #Resta 1
            if 0<p[int((len(p)-1)//2)]<=self.n-1: #Le resta uno al gen del medio
                p[int((len(p)-1)//2)] = p[int((len(p)-1)//2)] -1
            nueva_generacion.append(p)  
        return nueva_generacion #Devuelve la población mutada para el siguiente proceso
    
    def imprimir_optimo(self, pob_op):
        '''Función que se encarga de mostrar el individuo óptimo '''
        x_set = [] 
        for i,y in enumerate(pob_op):
            x_set.append(i) #Crea los valores para el eje x
        x = np.array(x_set) #Convierto en array para despúes poder agregarle 0.5 a cada valor
        path = 'img\queen2.png' #Nombre del ícono de corona
        fig, ax = plt.subplots()
        for xi, yi in zip(x_set, pob_op): #Agrega el ícono
            a = AnnotationBbox(OffsetImage(plt.imread(path), zoom= 0.4/self.n), (xi, yi), frameon=False)
            ax.add_artist(a)
        fig.set_size_inches(6, 6)#Configuración del tamaño del gráfico
        fig.set_dpi(100)
        plt.grid(True) #Habilita la grilla
        res = np.add.outer(range(self.n), range(self.n))%2 
        color_tablero = colors.ListedColormap(['maroon', 'tan']) #Color del tablero
        plt.imshow(res, cmap= color_tablero)
        plt.xticks(x+0.5) #Para que quede un punto en el medio
        plt.yticks(x+0.5)
        plt.xlim(-0.5, x[-1]+0.5) #Para que quede un punto en el medio
        plt.ylim(-0.5, x[-1]+0.5)
        plt.gca().set_aspect("equal") #Para que la grilla sea cuadrada
        plt.gca().axes.xaxis.set_ticklabels([]) #Para que no muestre los números
        plt.gca().axes.yaxis.set_ticklabels([])
        return 'parar' #Para decirle a la función recursiva que pare


### Código de la función fachada

In [ ]:
def encontrar_solucion():
    '''Función fachada para el usuario.
    Esta función ejecuta el proceso hasta encontrar una solución para la n ingresada. Si no encuentra la solución 
    para la población inicial en las primeras 2000 iteraciones, intenta con otra población. 
    Esto es para asegurar una respuesta, porque (no sé por qué) no me permite realizar más de 2000 interaciones con una recursiva.
    Ejemplo de uso:
    encontrar_solucion(4)
    '''
    try:
        n = int(input('Ingrese un número entre 4 y 8: '))
        if n<4: #Mensaje si se ingresa un número menor a 4
            return 'Entrada incorrecta. Ingrese un número entero mayor o igual 4 '
        else: 
            if n >=9: #Mensaje si se ingresa un número mayor a 9
                print('El número ingresado es mayor a 8. El programa puede tardar más en ejecutarse o generar errores. Se ejecutará de todas formas.')
            print('Ejecutando. Por favor, aguarde...')
            for i in range (1000): #Si en 2000 generaciones no encuentra el óptimo. Intenta con otra población inicial.
                a = ReinasN(n).generar_pob(n)
                if type(a) == str: 
                    break
    except ValueError: #Mensaje si se ingresa un valor incorrecto
        return 'Entrada incorrecta. Intente nuevamente ingresando un número entero.'
        

### Prueba

In [ ]:
'''Ejecutar'''
encontrar_solucion()

### Sitios consultados

* https://www.w3schools.com/python/numpy/numpy_array_slicing.asp
* https://www.w3schools.com/python/ref_random_shuffle.asp
* https://numpy.org/
* https://www.journaldev.com/32797/python-convert-numpy-array-to-list
* https://refactoring.guru/es/design-patterns/facade
* https://es.wikipedia.org/wiki/Problema_de_las_ocho_reinas
* https://stackoverflow.com/questions/22566284/matplotlib-how-to-plot-images-instead-of-points
* https://www.codespeedy.com/chess-board-using-matplotlib-python/
* https://stackoverflow.com/questions/9707676/defining-a-discrete-colormap-for-imshow-in-matplotlib